<a href="https://colab.research.google.com/github/Floral107/Mesterseges-intelligencia/blob/main/housing_lab5_hf_neuralis_halo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Feladat
Ebben a házi feladatban a cél házak árának predikciója lesz neurális hálózat segítségével. Az eredeti adathalmaz innen származik. Itt található róla egy rövid leírás. A neurális hálózat segítségével a környékre jellemző különböző 8 jellemző alapján (medián jövedelem, házak korának átlaga, szobák átlagos száma, stb.) a házak értékének mediánját kell megbecsülni. A cél, hogy minél pontosabb legyen a becslés, amit MSE (mean squared error) alapján határozunk meg.

Ehhez a megfelelő adathalmazt a moodle-ről lehet letölteni. Az adathalmaz linkje a moodle-ben a házi feladat alatt található a "Dataset assignment"-nél (ami egy 0 pontos quiz). Link a Moodle-höz
Ugyanitt olvasható egy "mse range" intervallum is, amely a maximális és minimális pontszámhoz tartozó MSE értékeket jelöli. A kapott pontszámot ezek között lineáris, ezen kívül pedig nearest neighbor interpolációval határozzuk meg.

A feladat tetszőleges környezetben megoldható. Mivel a területen a python a legelterjedtebb, ezért ehhez adunk segítséget.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


x_test = pd.read_csv(f'{"housing_x_test_70efdf"}.csv', sep=',', encoding='utf-8').values
x_train = pd.read_csv(f'{"housing_x_train_70efdf"}.csv', sep=',', encoding='utf-8').values
y_train = pd.read_csv(f'{"housing_y_train_70efdf"}.csv', sep=',', encoding='utf-8').values

scaler_x = MinMaxScaler().fit(x_train)
scaled_x_train = scaler_x.transform(x_train)
scaled_x_test = scaler_x.transform(x_test)

scaler_y = MinMaxScaler()
scaled_y_train = scaler_y.fit_transform(y_train)

X_train, X_val, y_train, y_val = train_test_split(scaled_x_train, scaled_y_train, test_size=0.2)

#model
def create_nn_model():
    model = tf.keras.Sequential()

    model.add(tf.keras.layers.Dense(128, activation="relu"))
    model.add(tf.keras.layers.Dense(128, activation="relu"))
    model.add(tf.keras.layers.Dropout(0.3))  # Dropout for regularization
    model.add(Dense(64, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.01)))  # L2 Regularization

    model.add(tf.keras.layers.Dense(1))
    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])
    return model

# Train two neural network models
model1 = create_nn_model()
model1.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), verbose=0)

model2 = create_nn_model()
model2.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), verbose=0)

# Korai leállás
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

# Tanulási ütem csökkentése
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=5, factor=0.5, verbose=1)

# Modell betanítása
history1 = model1.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val),
                    callbacks=[early_stopping, lr_scheduler])

y_test1 = model1.predict(scaled_x_test)

histor2 = model2.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val),
                    callbacks=[early_stopping, lr_scheduler])

y_test2 = model2.predict(scaled_x_test)

y_test = (y_test1 + y_test2) / 2
y_test_rescaled = scaler_y.inverse_transform(y_test)

print(y_test_rescaled)

np.savetxt('housing_y_test.csv', y_test_rescaled, delimiter=",", fmt="%g")




Epoch 1/100
358/358 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0113 - mean_squared_error: 0.0108 - val_loss: 0.0136 - val_mean_squared_error: 0.0131 - learning_rate: 0.0010
Epoch 2/100
358/358 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0113 - mean_squared_error: 0.0108 - val_loss: 0.0134 - val_mean_squared_error: 0.0129 - learning_rate: 0.0010
Epoch 3/100
358/358 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0119 - mean_squared_error: 0.0114 - val_loss: 0.0123 - val_mean_squared_error: 0.0118 - learning_rate: 0.0010
Epoch 4/100
358/358 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.0114 - mean_squared_error: 0.0109 - val_loss: 0.0125 - val_mean_squared_error: 0.0120 - learning_rate: 0.0010
Epoch 5/100
358/358 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0112 - mean_squared_error: 0.0107 - val_loss: 0.0121 - val_mean_squared_error: 0.0117 - learning_rate: 0.0010
Epoch 6/100
358/358 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0115 - mean_squared_error: 0.0110 - val_loss: 0.0124 - val_mean_squared_